## Desafio DWE 2025 - Carregamento e Transformação Camada Bronze
- Conversão dos arquivos da LandingZone que ainda estão em formato original para Delta Parquet

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("Carga e Transformação Bronze") \
    .config("spark.sql.shuffle.partitions", "200")  \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()


# Diretórios de trabalho 
lz_dim_path = '/mnt/desafio-dwe-25/lhdw/landingzone/dim'
lz_fato_path = '/mnt/desafio-dwe-25/lhdw/landingzone/fato'

lz_fato_in = f'{fato_path}/processar'
lz_fato_out = f'{fato_path}/processado'

lz_dim_in = f'{dim_path}/processar'
lz_dim_out = f'{dim_path}/processado'

br_dim_path = '/mnt/desafio-dwe-25/lhdw/bronze/dim'
br_fato_path = '/mnt/desafio-dwe-25/lhdw/bronze/fato'

### Dimensão - Categoria

In [0]:
lz_file = f'{lz_dim_in}/categorias.csv'

# Tipagem das Colunas
schema = StructType([
    StructField("CategoriaID", IntegerType(), True),
    StructField("NomeCategoria", StringType(), True)
])

df_categoria = spark.read \
    .format("csv") \
    .option("header", "true") \
    .schema(schema) \
    .load(lz_file)

df_categoria.show(5)

+-----------+-------------+
|CategoriaID|NomeCategoria|
+-----------+-------------+
|          1|  Confections|
|          2|   Shell fish|
|          3|      Cereals|
|          4|        Dairy|
|          5|    Beverages|
+-----------+-------------+
only showing top 5 rows



In [0]:
bronze_path = f'{br_dim_path}/CATEGORIA/'

# Conversão Delta Parquet
df_categoria.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(categoria_path)

spark.read.format("delta").load(categoria_path).show(5)


+-----------+-------------+
|CategoriaID|NomeCategoria|
+-----------+-------------+
|          1|  Confections|
|          2|   Shell fish|
|          3|      Cereals|
|          4|        Dairy|
|          5|    Beverages|
+-----------+-------------+
only showing top 5 rows



In [0]:
# Limpeza de cache e transferencia dos arquivos processados
dbutils.fs.mv(lz_file, lz_dim_out)
df_categoria.unpersist()

Out[27]: DataFrame[CategoriaID: int, NomeCategoria: string]

### Dimensão - Produto

In [0]:
lz_file = f'{lz_dim_in}/produtos.csv'

# Tipagem das Colunas
schema = StructType([
    StructField("ProdutoID", IntegerType(), True),
    StructField("ProdutoNome", StringType(), True),
    StructField("Preco", DoubleType(), True),
    StructField("CategoriaID", IntegerType(), True),
    StructField("Classe", StringType(), True),
    StructField("DataCadastro", StringType(), True),
    StructField("Resistencia", StringType(), True),
    StructField("EAlergico", StringType(), True),
    StructField("ValidadeDias", DoubleType(), True)
])

df_produto = spark.read \
    .format("csv") \
    .option("header", "true") \
    .schema(schema) \
    .load(lz_file)

df_produto.show(5)

+---------+--------------------+-------+-----------+------+--------------------+-----------+---------+------------+
|ProdutoID|         ProdutoNome|  Preco|CategoriaID|Classe|        DataCadastro|Resistencia|EAlergico|ValidadeDias|
+---------+--------------------+-------+-----------+------+--------------------+-----------+---------+------------+
|        1| Flour - Whole Wheat|74.2988|          3|Medium|2018-02-16 08:21:...|    Durable|  Unknown|         0.0|
|        2|Cookie Chocolate ...|91.2329|          3|Medium|2017-02-12 11:39:...|    Unknown|  Unknown|         0.0|
|        3|  Onions - Cippolini| 9.1379|          9|Medium|2018-03-15 08:11:...|       Weak|    False|       111.0|
|        4|Sauce - Gravy, Au...|54.3055|          9|Medium|2017-07-16 00:46:...|    Durable|  Unknown|         0.0|
|        5|Artichokes - Jeru...|65.4771|          2|   Low|2017-08-16 14:13:...|    Durable|     True|        27.0|
+---------+--------------------+-------+-----------+------+-------------

In [0]:
bronze_path = f'{br_dim_path}/PRODUTO/'

# Conversão Delta Parquet
df_produto.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(bronze_path)

spark.read.format("delta").load(bronze_path).show(5)

+---------+--------------------+-------+-----------+------+--------------------+-----------+---------+------------+
|ProdutoID|         ProdutoNome|  Preco|CategoriaID|Classe|        DataCadastro|Resistencia|EAlergico|ValidadeDias|
+---------+--------------------+-------+-----------+------+--------------------+-----------+---------+------------+
|        1| Flour - Whole Wheat|74.2988|          3|Medium|2018-02-16 08:21:...|    Durable|  Unknown|         0.0|
|        2|Cookie Chocolate ...|91.2329|          3|Medium|2017-02-12 11:39:...|    Unknown|  Unknown|         0.0|
|        3|  Onions - Cippolini| 9.1379|          9|Medium|2018-03-15 08:11:...|       Weak|    False|       111.0|
|        4|Sauce - Gravy, Au...|54.3055|          9|Medium|2017-07-16 00:46:...|    Durable|  Unknown|         0.0|
|        5|Artichokes - Jeru...|65.4771|          2|   Low|2017-08-16 14:13:...|    Durable|     True|        27.0|
+---------+--------------------+-------+-----------+------+-------------

In [0]:
# Limpeza de cache e transferencia dos arquivos processados
dbutils.fs.mv(lz_file, lz_dim_out)
df_produto.unpersist()

Out[41]: DataFrame[ProdutoID: int, ProdutoNome: string, Preco: double, CategoriaID: int, Classe: string, DataCadastro: string, Resistencia: string, EAlergico: string, ValidadeDias: double]

### Dimensão - Vendedor

In [0]:
lz_file = f'{lz_dim_in}/vendedores.csv'

# Tipagem das Colunas
schema = StructType([
    StructField("VendedorID", IntegerType(), True),
    StructField("PrimeiroNome", StringType(), True),
    StructField("NomeDoMeio", StringType(), True),
    StructField("UltimoNome", StringType(), True),
    StructField("DataNascimento", DateType(), True),
    StructField("Genero", StringType(), True),
    StructField("CidadeID", IntegerType(), True),
    StructField("DataAdmissao", DateType(), True)
])

df_vendedor = spark.read \
    .format("csv") \
    .option("header", "true") \
    .schema(schema) \
    .load(lz_file)

df_vendedor.show(5)

+----------+------------+----------+----------+--------------+------+--------+------------+
|VendedorID|PrimeiroNome|NomeDoMeio|UltimoNome|DataNascimento|Genero|CidadeID|DataAdmissao|
+----------+------------+----------+----------+--------------+------+--------+------------+
|         1|      Nicole|         T|    Fuller|    1981-03-07|     F|      80|  2011-06-20|
|         2|   Christine|         W|    Palmer|    1968-01-25|     F|       4|  2011-04-27|
|         3|       Pablo|         Y|     Cline|    1963-02-09|     M|      70|  2012-03-30|
|         4|     Darnell|         O|   Nielsen|    1989-02-06|     M|      39|  2014-03-06|
|         5|     Desiree|         L|    Stuart|    1963-05-03|     F|      23|  2014-11-16|
+----------+------------+----------+----------+--------------+------+--------+------------+
only showing top 5 rows



In [0]:
bronze_path = f'{br_dim_path}/VENDEDOR/'

# Conversão Delta Parquet
df_vendedor.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(bronze_path)

spark.read.format("delta").load(bronze_path).show(5)

+----------+------------+----------+----------+--------------+------+--------+------------+
|VendedorID|PrimeiroNome|NomeDoMeio|UltimoNome|DataNascimento|Genero|CidadeID|DataAdmissao|
+----------+------------+----------+----------+--------------+------+--------+------------+
|         1|      Nicole|         T|    Fuller|    1981-03-07|     F|      80|  2011-06-20|
|         2|   Christine|         W|    Palmer|    1968-01-25|     F|       4|  2011-04-27|
|         3|       Pablo|         Y|     Cline|    1963-02-09|     M|      70|  2012-03-30|
|         4|     Darnell|         O|   Nielsen|    1989-02-06|     M|      39|  2014-03-06|
|         5|     Desiree|         L|    Stuart|    1963-05-03|     F|      23|  2014-11-16|
+----------+------------+----------+----------+--------------+------+--------+------------+
only showing top 5 rows



In [0]:
# Limpeza de cache e transferencia dos arquivos processados
dbutils.fs.mv(lz_file, lz_dim_out)
df_vendedor.unpersist()

Out[40]: DataFrame[VendedorID: int, PrimeiroNome: string, NomeDoMeio: string, UltimoNome: string, DataNascimento: date, Genero: string, CidadeID: int, DataAdmissao: date]

### Dimensão - País

In [0]:
lz_file = f'{lz_dim_in}/paises.csv'

# Tipagem das Colunas
schema = StructType([
    StructField("PaisID", IntegerType(), True),
    StructField("PaisNome", StringType(), True),
    StructField("SiglaPais", StringType(), True)
])

df_pais = spark.read \
    .format("csv") \
    .option("header", "true") \
    .schema(schema) \
    .load(lz_file)

df_pais.show(5)

+------+--------+---------+
|PaisID|PaisNome|SiglaPais|
+------+--------+---------+
|     1| Armenia|       AN|
|     2|  Canada|       FO|
|     3|  Belize|       MK|
|     4|  Uganda|       LV|
|     5|Thailand|       VI|
+------+--------+---------+
only showing top 5 rows



In [0]:
bronze_path = f'{br_dim_path}/PAIS/'

# Conversão Delta Parquet
df_pais.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(bronze_path)

spark.read.format("delta").load(bronze_path).show(5)

+------+--------+---------+
|PaisID|PaisNome|SiglaPais|
+------+--------+---------+
|     1| Armenia|       AN|
|     2|  Canada|       FO|
|     3|  Belize|       MK|
|     4|  Uganda|       LV|
|     5|Thailand|       VI|
+------+--------+---------+
only showing top 5 rows



In [0]:
# Limpeza de cache e transferencia dos arquivos processados
dbutils.fs.mv(lz_file, lz_dim_out)
df_pais.unpersist()

Out[39]: DataFrame[PaisID: int, PaisNome: string, SiglaPais: string]

### Dimensão - Cidade

In [0]:
lz_file = f'{lz_dim_in}/cidades.csv'

# Tipagem das Colunas
schema = StructType([
    StructField("CidadeID", IntegerType(), True),
    StructField("NomeCidade", StringType(), True),
    StructField("Cep", StringType(), True),
    StructField("PaisID", StringType(), True)
])

df_cidade = spark.read \
    .format("csv") \
    .option("header", "true") \
    .schema(schema) \
    .load(lz_file)

df_cidade.show(5)

+--------+--------------+-----+------+
|CidadeID|    NomeCidade|  Cep|PaisID|
+--------+--------------+-----+------+
|       1|        Dayton|80563|    32|
|       2|       Buffalo|17420|    32|
|       3|       Chicago|44751|    32|
|       4|       Fremont|20641|    32|
|       5|Virginia Beach|62389|    32|
+--------+--------------+-----+------+
only showing top 5 rows



In [0]:
bronze_path = f'{br_dim_path}/CIDADE/'

# Conversão Delta Parquet
df_cidade.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(bronze_path)

spark.read.format("delta").load(bronze_path).show(5)

+--------+--------------+-----+------+
|CidadeID|    NomeCidade|  Cep|PaisID|
+--------+--------------+-----+------+
|       1|        Dayton|80563|    32|
|       2|       Buffalo|17420|    32|
|       3|       Chicago|44751|    32|
|       4|       Fremont|20641|    32|
|       5|Virginia Beach|62389|    32|
+--------+--------------+-----+------+
only showing top 5 rows



In [0]:
# Limpeza de cache e transferencia dos arquivos processados
dbutils.fs.mv(lz_file, lz_dim_out)
df_cidade.unpersist()

Out[44]: DataFrame[CidadeID: int, NomeCidade: string, Cep: string, PaisID: string]

### Dimensão - Cliente

In [0]:
lz_file = f'{lz_dim_in}/clientes.csv'

# Tipagem das Colunas
schema = StructType([
    StructField("ClienteID", IntegerType(), True),
    StructField("PrimeiroNome", StringType(), True),
    StructField("NomeDoMeio", StringType(), True),
    StructField("UltimoNome", StringType(), True),
    StructField("CidadeID", IntegerType(), True),
    StructField("Endereco", StringType(), True)
])

df_cliente = spark.read \
    .format("csv") \
    .option("header", "true") \
    .schema(schema) \
    .load(lz_file)

df_cliente.show(5)

+---------+------------+----------+----------+--------+--------------------+
|ClienteID|PrimeiroNome|NomeDoMeio|UltimoNome|CidadeID|            Endereco|
+---------+------------+----------+----------+--------+--------------------+
|        1|    Stefanie|         Y|      Frye|      79|       97 Oak Avenue|
|        2|       Sandy|         T|     Kirby|      96|52 White First Fr...|
|        3|         Lee|         T|     Zhang|      55|921 White Fabien ...|
|        4|      Regina|         S|     Avery|      40|       75 Old Avenue|
|        5|      Daniel|         S|    Mccann|       2|283 South Green H...|
+---------+------------+----------+----------+--------+--------------------+
only showing top 5 rows



In [0]:
bronze_path = f'{br_dim_path}/CLIENTE/'

# Conversão Delta Parquet
df_cliente.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(bronze_path)

spark.read.format("delta").load(bronze_path).show(5)

+---------+------------+----------+----------+--------+--------------------+
|ClienteID|PrimeiroNome|NomeDoMeio|UltimoNome|CidadeID|            Endereco|
+---------+------------+----------+----------+--------+--------------------+
|        1|    Stefanie|         Y|      Frye|      79|       97 Oak Avenue|
|        2|       Sandy|         T|     Kirby|      96|52 White First Fr...|
|        3|         Lee|         T|     Zhang|      55|921 White Fabien ...|
|        4|      Regina|         S|     Avery|      40|       75 Old Avenue|
|        5|      Daniel|         S|    Mccann|       2|283 South Green H...|
+---------+------------+----------+----------+--------+--------------------+
only showing top 5 rows



In [0]:
# Limpeza de cache e transferencia dos arquivos processados
dbutils.fs.mv(lz_file, lz_dim_out)
df_cliente.unpersist()

Out[47]: DataFrame[ClienteID: int, PrimeiroNome: string, NomeDoMeio: string, UltimoNome: string, CidadeID: int, Endereco: string]

### Fato - Vendas

In [0]:
lz_file = f'{lz_fato_in}/'

# Tipagem das Colunas
schema = StructType([
    StructField("VendasID", IntegerType(), True),
    StructField("VendedorID", IntegerType(), True),
    StructField("ClienteID", IntegerType(), True),
    StructField("ProdutoID", IntegerType(), True),
    StructField("Quantidade", IntegerType(), True),
    StructField("Desconto", DoubleType(), True),
    StructField("PrecoTotal", DoubleType(), True),
    StructField("DataVenda", DateType(), True),
    StructField("NumeroTransacao", StringType(), True)
])

df_vendas = spark.read \
    .format("csv") \
    .option("header", "true") \
    .schema(schema) \
    .load(lz_file) \
    .withColumn(
        "filename", 
        regexp_extract(input_file_name(), "([^/]+)$", 0)
    )    

df_vendas.show(5)

+--------+----------+---------+---------+----------+--------+----------+----------+--------------------+------------+
|VendasID|VendedorID|ClienteID|ProdutoID|Quantidade|Desconto|PrecoTotal| DataVenda|     NumeroTransacao|    filename|
+--------+----------+---------+---------+----------+--------+----------+----------+--------------------+------------+
| 6207624|        16|     1401|        7|         1|     0.1|       0.0|2018-01-01|JMAICVALK5QLQPFJZMPQ|vendas_1.csv|
| 2266176|         1|    28628|       79|         8|     0.1|       0.0|2018-01-01|XJ2AASIGXZFUOKQ1HMLJ|vendas_1.csv|
| 6510351|        12|    64721|      411|        17|     0.0|       0.0|2018-01-01|AISH11C7JF8C4YRZ7PLQ|vendas_1.csv|
| 1972817|         3|    84022|      425|        22|     0.0|       0.0|2018-01-01|SQQ4UEB0ML0QPNLTNLG9|vendas_1.csv|
| 5564470|        17|    10737|      101|         3|     0.0|       0.0|2018-01-01|1AVUBWY1L2JL8ZXG5KE2|vendas_1.csv|
+--------+----------+---------+---------+----------+----

In [0]:
df_vendas.groupBy("filename").count().show()

+------------+------+
|    filename| count|
+------------+------+
|vendas_1.csv|321815|
|vendas_2.csv|321815|
|vendas_3.csv|321816|
+------------+------+



In [0]:
bronze_path = f'{br_fato_path}/VENDAS/'

# Limpeza da coluna de verificação
df_vendas = df_vendas.drop("filename")

# Conversão Delta Parquet
df_vendas.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(bronze_path)

spark.read.format("delta").load(bronze_path).show(5)

+--------+----------+---------+---------+----------+--------+----------+----------+--------------------+
|VendasID|VendedorID|ClienteID|ProdutoID|Quantidade|Desconto|PrecoTotal| DataVenda|     NumeroTransacao|
+--------+----------+---------+---------+----------+--------+----------+----------+--------------------+
| 6207624|        16|     1401|        7|         1|     0.1|       0.0|2018-01-01|JMAICVALK5QLQPFJZMPQ|
| 2266176|         1|    28628|       79|         8|     0.1|       0.0|2018-01-01|XJ2AASIGXZFUOKQ1HMLJ|
| 6510351|        12|    64721|      411|        17|     0.0|       0.0|2018-01-01|AISH11C7JF8C4YRZ7PLQ|
| 1972817|         3|    84022|      425|        22|     0.0|       0.0|2018-01-01|SQQ4UEB0ML0QPNLTNLG9|
| 5564470|        17|    10737|      101|         3|     0.0|       0.0|2018-01-01|1AVUBWY1L2JL8ZXG5KE2|
+--------+----------+---------+---------+----------+--------+----------+----------+--------------------+
only showing top 5 rows



In [0]:
# Limpeza de cache e transferencia dos arquivos processados
dbutils.fs.mv(lz_file, lz_fato_out, recurse=True)
df_vendas.unpersist()

Out[59]: DataFrame[VendasID: int, VendedorID: int, ClienteID: int, ProdutoID: int, Quantidade: int, Desconto: double, PrecoTotal: double, DataVenda: date, NumeroTransacao: string]

### Analise da Carga - Fato

In [0]:
bronze_path = f'{br_fato_path}/VENDAS'

df_fato = spark.read.format("delta").load(bronze_path)

# Exibindo sumário
print(f"Linhas: {df_fato.count()}")
print(f"Colunas: {df_fato.columns}")
df_fato.summary().show(10)

Linhas: 965446
Colunas: ['VendasID', 'VendedorID', 'ClienteID', 'ProdutoID', 'Quantidade', 'Desconto', 'PrecoTotal', 'DataVenda', 'NumeroTransacao']
+-------+------------------+------------------+------------------+------------------+------------------+--------------------+----------+--------------------+
|summary|          VendasID|        VendedorID|         ClienteID|         ProdutoID|        Quantidade|            Desconto|PrecoTotal|     NumeroTransacao|
+-------+------------------+------------------+------------------+------------------+------------------+--------------------+----------+--------------------+
|  count|            965446|            965446|            965446|            965446|            965446|              965446|    965446|              965446|
|   mean|3378612.8861583145|11.993800792587052|  49402.5070589137|226.58830737296546|13.005468974960795|0.029894370063150925|       0.0|                null|
| stddev|1951396.9390423202| 6.629074016941077|28525.08856865